In [1]:
import pandas as pd
import os
import json

In [ ]:
import os
import pandas as pd
import json

def procesar_archivo_json(archivo_json, tamano_parte):
    with open(archivo_json, 'r', encoding="utf-8") as file:
        parte_actual = []
        num_parte = 1
        
        for linea in file:
            objeto_json = json.loads(linea)
            parte_actual.append(objeto_json)
            
            if len(parte_actual) >= tamano_parte:
                # Procesar la parte actual
                procesar_y_guardar_parte(parte_actual, num_parte, archivo_json)
                parte_actual = []
                num_parte += 1
                
        # Procesar la última parte si quedan datos
        if parte_actual:
            procesar_y_guardar_parte(parte_actual, num_parte, archivo_json)

def procesar_archivo_parquet(archivo_parquet, tamano_parte):
    parte_actual = []
    num_parte = 1
    
    # Leer el archivo Parquet en partes
    for df_parte in pd.read_parquet(archivo_parquet, chunksize=tamano_parte):
        parte_actual.append(df_parte)
        
        if len(parte_actual) >= tamano_parte:
            # Procesar y guardar la parte actual
            procesar_y_guardar_parte(parte_actual, num_parte, archivo_parquet)
            parte_actual = []
            num_parte += 1
            
    # Procesar y guardar la última parte si quedan datos
    if parte_actual:
        procesar_y_guardar_parte(parte_actual, num_parte, archivo_parquet)

def procesar_y_guardar_parte(parte, num_parte, archivo):
    dfs_parte = []
    for item in parte:
        if isinstance(item, dict):
            dfs_parte.append(pd.DataFrame([item]))
        elif isinstance(item, pd.DataFrame):
            dfs_parte.append(item)
        else:
            print(f"Elemento no válido en la parte {num_parte}: {item}")
    
    # Concatenar los DataFrames en un solo DataFrame si hay alguno
    if dfs_parte:
        df = pd.concat(dfs_parte, ignore_index=True)
        
        # Determinar el nombre del archivo de salida y la ruta de destino
        nombre_archivo = os.path.splitext(os.path.basename(archivo))[0]
        ruta_destino = os.path.dirname(archivo)
        
        # Guardar la parte procesada en un archivo JSON o Parquet según la extensión del archivo original
        if archivo.endswith(".json"):
            df.to_json(os.path.join(ruta_destino, f"{nombre_archivo}_parte_{num_parte}.json"), index=False)
        elif archivo.endswith(".parquet"):
            df.to_parquet(os.path.join(ruta_destino, f"{nombre_archivo}_parte_{num_parte}.parquet"), index=False)

# Directorio donde se encuentran los archivos JSON
directorio_origen = r'C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP'
tamano_parte = 200000  # Número máximo de registros por parte

# Recorrer cada archivo en el directorio de origen
for archivo in os.listdir(directorio_origen):
    #if archivo.endswith(".json"):
    #    archivo_json = os.path.join(directorio_origen, archivo)
    #    procesar_archivo_json(archivo_json, tamano_parte)
    if archivo.endswith(".parquet"):
        archivo_parquet = os.path.join(directorio_origen, archivo)
        procesar_archivo_parquet(archivo_parquet, tamano_parte)

    

In [10]:
import os
import pandas as pd

def procesar_archivo_parquet(archivo_parquet, tamano_parte):
    df = pd.read_parquet(archivo_parquet)
    total_registros = len(df)
    num_parte = 1
    inicio = 0
    
    while inicio < total_registros:
        fin = min(inicio + tamano_parte, total_registros)
        parte_actual = df.iloc[inicio:fin]
        procesar_y_guardar_parte(parte_actual, num_parte, archivo_parquet)
        num_parte += 1
        inicio = fin

def procesar_y_guardar_parte(parte, num_parte, archivo):
    # Determinar el nombre del archivo de salida y la ruta de destino
    nombre_archivo = os.path.splitext(os.path.basename(archivo))[0]
    ruta_destino = os.path.dirname(archivo)
    
    # Guardar la parte procesada en un archivo Parquet
    parte.to_json(os.path.join(ruta_destino, f"{nombre_archivo}_parte_{num_parte}.json"), index=False)

# Directorio donde se encuentran los archivos Parquet
directorio_origen = r'C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP'
tamano_parte = 100000  # Número máximo de registros por parte

# Recorrer cada archivo en el directorio de origen
for archivo in os.listdir(directorio_origen):
    if archivo.endswith(".parquet"):
        archivo_parquet = os.path.join(directorio_origen, archivo)
        procesar_archivo_parquet(archivo_parquet, tamano_parte)


In [ ]:
import functions_framework
import json
import pandas as pd 
import numpy as np 
from google.cloud import bigquery
from google.cloud import storage

# Importar modulo processing
#from processing import procesar, hash_category

# Instatiate clients
cloud_storage = storage.Client()
bq = bigquery.Client()

project_id = 'dms-pfh'

def archivo_df_business(file_uri):
    # Crear una instancia del cliente de Google Cloud Storage
    client = storage.Client()

    # Nombre del archivo en el bucket
    nombre_archivo = 'df_mascara.csv'

    # Nombre del bucket
    nombre_bucket = 'yelp_dms'
    # Ruta completa en el bucket
    ruta_archivo = f'gs://{nombre_bucket}/{nombre_archivo}'
    
    # Filtrado segun nombre archivo para su respectivo ETL
    df_bussines = pd.read_pickle(file_uri)
    df_bussines =df_bussines.loc[:,~df_bussines.columns.duplicated()].copy()
    print("pase la mascara de juan de columnas")
    df_bussines.drop(columns=["is_open",'attributes', 'hours','postal_code'], inplace=True)
    categorias_deseadas = ["restaurant", "bar", "pub", "cafe", 'bakery', 'coffee shop', 'gym', 'gas station']

    # Eliminar filas con valores NaN en la columna 'categories'
    df_bussines = df_bussines.fillna("No Data")
    # Filtrar el DataFrame para mantener solo las filas que contienen al menos una de las categorías deseadas
    df_bussines = df_bussines[df_bussines['categories'].str.contains('|'.join(categorias_deseadas), case=False)]
    df_mascara= df_bussines["business_id"]
    

    # Guardar el DataFrame como un archivo CSV
    df_mascara.to_csv(nombre_archivo, index=False)
    # Obtener el bucket
    bucket = client.bucket(nombre_bucket)
    # Subir el archivo al bucket
    blob = bucket.blob(nombre_archivo)
    blob.upload_from_filename(nombre_archivo)
    print(f'DataFrame guardado en {ruta_archivo}')
    return df_bussines

def archivo_df_tip(file_uri):
    df_tip = pd.read_json(file_uri, lines=True)
    df_tip.drop(columns="compliment_count", inplace=True)
    for columna in df_tip.columns:
        if df_tip[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_tip[columna] = df_tip[columna].fillna("No data").str.strip()
        elif df_tip[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_tip[columna] = df_tip[columna].fillna(0)
    return df_tip

def archivo_df_user(file_uri):
    df_user001 = pd.read_json(file_uri, lines=True)
    df_user001.drop(columns=['elite','friends','compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute','compliment_list', 'compliment_note', 'compliment_plain',
        'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos'], inplace=True)
    for columna in df_user001.columns:
        if df_user001[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_user001[columna] = df_user001[columna].fillna("No data").str.strip()
        elif df_user001[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_user001[columna] = df_user001[columna].fillna(0)
    print("completado etl archivo parquet")
    return df_user001

def archivo_df_review(file_uri):
    df_review = pd.read_json(file_uri, lines=True)
    for columna in df_review.columns:
        #if df_review[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
        #    df_review[columna] = df_review[columna].fillna("No data").str.strip()
        if df_review[columna].dtype in ['int64', 'float64']:
                    # Si la columna es numérica, llenar los valores nulos con 0
            df_review[columna] = df_review[columna].fillna(0)
    df_mascara=pd.read_csv("gs://yelp_dms/df_mascara.csv")
    print("carga de mascara completada")
    df_review = pd.merge(df_review, df_mascara, on='business_id', how='inner')
    print("filtracion completa")
    return df_review

# Triggered by a change in a storage bucket
@functions_framework.cloud_event
def load_df(cloud_event):
    data = cloud_event.data
    event_id = cloud_event["id"]
    event_type = cloud_event["type"]
    print(f'Cloud event ID: {event_id}')
    print(f'Event type: {event_type}')
    bucket_name = data["bucket"]
    name = data["name"]
    print(f'Bucket: {bucket_name}')
    print(f'File name: {name}')
    # Create URI
    file_uri = f'gs://{bucket_name}/{name}'
    print(f'URI: {file_uri}')
    # Obtener el objeto del bucket
    bucket = cloud_storage.bucket(bucket_name)
    # Obtener el blob (objeto) del archivo JSON en el bucket
    blob = bucket.blob(name)
    # Attempt to create DataFrame with URI
    # Attempt to process DataFrame
    print('Transformando dataframe...')
    try:
        if "business" in file_uri:
            df_procesado = archivo_df_business(file_uri)
            table_name = "yelp-sites"
            print("dataframe cargado") 
        if "tip" in file_uri:
            df_procesado = archivo_df_tip(file_uri)
            table_name = "yelp-user-tips"
            print("dataframe cargado")
        if "user-001" in file_uri:
            df_procesado = archivo_df_user(file_uri)
            table_name = "yelp-user-info"
            print("dataframe USER-001 cargado")
        if "review-002" in file_uri:
            df_procesado=archivo_df_review(file_uri)
            table_name="yelp-user-review"
            print("dataframe REVIEW-002 cargado")
        print('DataFrame Procesado')
        print('Nuevas dimensiones:', df_procesado.shape)
    except Exception as e:
        # Cuando no procesa la información
        print('DataFrame no pudo ser procesado: Error - ', e)
        return None
    # Insert to table
    table_id = f'dms-pfh.yelp_data.{table_name}'
    # Create job configurations
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        create_disposition='CREATE_IF_NEEDED',
        write_disposition='WRITE_APPEND'
        # source_format='NEWLINE_DELIMITED_JSON'
    )
    print('Job configuration created.')
    # Requesting the job
    load_job = bq.load_table_from_dataframe(
        df_procesado,
        table_id,
        location='us-central1',
        job_config=job_config
    )
    load_job.result()
    print('Job finished')


In [11]:
df_aux=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002_parte_1.json", lines=True)

In [14]:
df_aux['business_id'] = df_aux['business_id'].astype(str)


# Verificar el cambio
print(df_aux.dtypes)

review_id      object
user_id        object
business_id    object
stars          object
useful         object
funny          object
cool           object
text           object
date           object
dtype: object


In [2]:
df_mascara=pd.read_csv(r"C:\Users\Usuario\Desktop\GitHub\df_mascara(1).csv")

In [10]:
df_aux2=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002_parte_2.json", lines=True)
df_aux3=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002_parte_3.json", lines=True)
df_aux4=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002_parte_4.json", lines=True)
df_aux2['business_id'] = df_aux2['business_id'].astype(str)
df_aux2['business_id'] = df_aux2['business_id'].str.strip()
df_aux3['business_id'] = df_aux3['business_id'].astype(str)
df_aux3['business_id'] = df_aux3['business_id'].str.strip()
df_aux4['business_id'] = df_aux4['business_id'].astype(str)
df_aux4['business_id'] = df_aux4['business_id'].str.strip()

In [15]:
df_review = pd.merge(df_aux4, df_mascara, on='business_id', how='inner')

In [20]:
df_review=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002.json",lines=True, chunksize=100)

In [38]:
import pandas as pd
import json
import os

archivo_json = r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\review-002.json"
chunksize = 100000  # Tamaño de cada parte
output_directory = r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\partes"  # Directorio de salida

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Leer el archivo JSON en partes más pequeñas
partes = pd.read_json(archivo_json, lines=True, chunksize=chunksize)

# Procesar cada parte por separado
for i, parte in enumerate(partes):
    # Determinar el nombre del archivo de salida
    nombre_archivo = f"parte_{i+1}.json"
    ruta_archivo = os.path.join(output_directory, nombre_archivo)
    
    # Guardar la parte como archivo JSON
    parte.to_json(ruta_archivo, orient="records", lines=True)
    
    print(f"Parte {i+1} guardada como {nombre_archivo}")





Parte 1 guardada como parte_1.json
Parte 2 guardada como parte_2.json
Parte 3 guardada como parte_3.json
Parte 4 guardada como parte_4.json
Parte 5 guardada como parte_5.json
Parte 6 guardada como parte_6.json
Parte 7 guardada como parte_7.json
Parte 8 guardada como parte_8.json
Parte 9 guardada como parte_9.json
Parte 10 guardada como parte_10.json
Parte 11 guardada como parte_11.json
Parte 12 guardada como parte_12.json
Parte 13 guardada como parte_13.json
Parte 14 guardada como parte_14.json
Parte 15 guardada como parte_15.json
Parte 16 guardada como parte_16.json
Parte 17 guardada como parte_17.json
Parte 18 guardada como parte_18.json
Parte 19 guardada como parte_19.json
Parte 20 guardada como parte_20.json
Parte 21 guardada como parte_21.json
Parte 22 guardada como parte_22.json
Parte 23 guardada como parte_23.json
Parte 24 guardada como parte_24.json
Parte 25 guardada como parte_25.json
Parte 26 guardada como parte_26.json
Parte 27 guardada como parte_27.json
Parte 28 guardada c

In [40]:
df_parte=pd.read_json(r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\partes\parte_1.json", lines=True)

particion user

In [46]:
import pandas as pd
import os

archivo_parquet = r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP\user-001.parquet"
chunksize = 100000  # Tamaño de cada parte
output_directory = r"C:\Users\Usuario\Desktop\GitHub\DataSets\DataLake\YELP"  # Directorio de salida

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Leer el archivo Parquet completo
df = pd.read_parquet(archivo_parquet)

# Dividir el DataFrame en partes más pequeñas
num_partes = len(df) // chunksize + 1
for i, start_idx in enumerate(range(0, len(df), chunksize)):
    # Determinar el nombre del archivo de salida
    nombre_archivo = f"user_{i+1}.json"
    ruta_archivo = os.path.join(output_directory, nombre_archivo)
    
    # Obtener la parte actual del DataFrame
    parte = df.iloc[start_idx:start_idx+chunksize]
    
    # Guardar la parte como archivo JSON
    parte.to_json(ruta_archivo, orient="records", lines=True)
    
    print(f"Parte {i+1} guardada como {nombre_archivo}")



Parte 1 guardada como user_1.json
Parte 2 guardada como user_2.json
Parte 3 guardada como user_3.json
Parte 4 guardada como user_4.json
Parte 5 guardada como user_5.json
Parte 6 guardada como user_6.json
Parte 7 guardada como user_7.json
Parte 8 guardada como user_8.json
Parte 9 guardada como user_9.json
Parte 10 guardada como user_10.json
Parte 11 guardada como user_11.json
Parte 12 guardada como user_12.json
Parte 13 guardada como user_13.json
Parte 14 guardada como user_14.json
Parte 15 guardada como user_15.json
Parte 16 guardada como user_16.json
Parte 17 guardada como user_17.json
Parte 18 guardada como user_18.json
Parte 19 guardada como user_19.json
Parte 20 guardada como user_20.json
Parte 21 guardada como user_21.json
Parte 22 guardada como user_22.json


In [ ]:
import functions_framework
import json
import pandas as pd 
import numpy as np 
from google.cloud import bigquery
from google.cloud import storage

# Importar modulo processing
#from processing import procesar, hash_category

# Instatiate clients
cloud_storage = storage.Client()
bq = bigquery.Client()

project_id = 'dms-pfh'

def archivo_df_business(file_uri):
    # Crear una instancia del cliente de Google Cloud Storage
    client = storage.Client()

    # Nombre del archivo en el bucket
    nombre_archivo = 'df_mascara.csv'

    # Nombre del bucket
    nombre_bucket = 'yelp_dms'
    # Ruta completa en el bucket
    ruta_archivo = f'gs://{nombre_bucket}/{nombre_archivo}'
    
    # Filtrado segun nombre archivo para su respectivo ETL
    df_bussines = pd.read_pickle(file_uri)
    df_bussines =df_bussines.loc[:,~df_bussines.columns.duplicated()].copy()
    print("pase la mascara de juan de columnas")
    df_bussines.drop(columns=["is_open",'attributes', 'hours','postal_code'], inplace=True)
    categorias_deseadas = ["restaurant", "bar", "pub", "cafe", 'bakery', 'coffee shop', 'gym', 'gas station']

    # Eliminar filas con valores NaN en la columna 'categories'
    df_bussines = df_bussines.fillna("No Data")
    # Filtrar el DataFrame para mantener solo las filas que contienen al menos una de las categorías deseadas
    df_bussines = df_bussines[df_bussines['categories'].str.contains('|'.join(categorias_deseadas), case=False)]
    df_mascara= df_bussines["business_id"]
    

    # Guardar el DataFrame como un archivo CSV
    df_mascara.to_csv(nombre_archivo, index=False)
    # Obtener el bucket
    bucket = client.bucket(nombre_bucket)
    # Subir el archivo al bucket
    blob = bucket.blob(nombre_archivo)
    blob.upload_from_filename(nombre_archivo)
    print(f'DataFrame guardado en {ruta_archivo}')
    return df_bussines

def archivo_df_tip(file_uri):
    df_tip = pd.read_json(file_uri, lines=True)
    df_tip.drop(columns="compliment_count", inplace=True)
    for columna in df_tip.columns:
        if df_tip[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_tip[columna] = df_tip[columna].fillna("No data").str.strip()
        elif df_tip[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_tip[columna] = df_tip[columna].fillna(0)
    return df_tip

def archivo_df_user(file_uri):
    df_user001 = pd.read_json(file_uri, lines=True)
    df_user001.drop(columns=['elite','friends','compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute','compliment_list', 'compliment_note', 'compliment_plain',
        'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos'], inplace=True)
    for columna in df_user001.columns:
        if df_user001[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_user001[columna] = df_user001[columna].fillna("No data").str.strip()
        elif df_user001[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_user001[columna] = df_user001[columna].fillna(0)
    print("completado etl archivo parquet")
    return df_user001

def archivo_df_review(file_uri):
    df_review = pd.read_json(file_uri, lines=True)
    for columna in df_review.columns:
        #if df_review[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
        #    df_review[columna] = df_review[columna].fillna("No data").str.strip()
        if df_review[columna].dtype in ['int64', 'float64']:
                    # Si la columna es numérica, llenar los valores nulos con 0
            df_review[columna] = df_review[columna].fillna(0)
    df_mascara=pd.read_csv("gs://yelp_dms/df_mascara.csv")
    print("carga de mascara completada")
    df_review['business_id'] = df_review['business_id'].astype(str)
    df_mascara['business_id'] = df_mascara['business_id'].astype(str)

    df_review = pd.merge(df_review, df_mascara, on='business_id', how='inner')
    print("filtracion completa en archivo REVIEW")
    return df_review

# Triggered by a change in a storage bucket
@functions_framework.cloud_event
def load_df(cloud_event):
    data = cloud_event.data
    event_id = cloud_event["id"]
    event_type = cloud_event["type"]
    print(f'Cloud event ID: {event_id}')
    print(f'Event type: {event_type}')
    bucket_name = data["bucket"]
    name = data["name"]
    print(f'Bucket: {bucket_name}')
    print(f'File name: {name}')
    # Create URI
    file_uri = f'gs://{bucket_name}/{name}'
    print(f'URI: {file_uri}')
    # Obtener el objeto del bucket
    bucket = cloud_storage.bucket(bucket_name)
    # Obtener el blob (objeto) del archivo JSON en el bucket
    blob = bucket.blob(name)
    # Attempt to create DataFrame with URI
    # Attempt to process DataFrame
    print('Transformando dataframe...')
    try:
        if "business" in file_uri:
            df_procesado = archivo_df_business(file_uri)
            table_name = "yelp-sites"
            print("dataframe cargado") 
        if "tip" in file_uri:
            df_procesado = archivo_df_tip(file_uri)
            table_name = "yelp-user-tips"
            print("dataframe cargado")
        if "user-001" in file_uri:
            df_procesado = archivo_df_user(file_uri)
            table_name = "yelp-user-info"
            print("dataframe cargado")
        if "review-002" in file_uri:
            df_procesado=archivo_df_review(file_uri)
            table_name="yelp-user-review"
            print("dataframe cargado")
        print('DataFrame Procesado')
        print('Nuevas dimensiones:', df_procesado.shape)
    except Exception as e:
        # Cuando no procesa la información
        print('DataFrame no pudo ser procesado: Error - ', e)
        return None
    # Insert to table
    table_id = f'dms-pfh.yelp_data.{table_name}'
    # Create job configurations
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        create_disposition='CREATE_IF_NEEDED',
        write_disposition='WRITE_APPEND'
        # source_format='NEWLINE_DELIMITED_JSON'
    )
    print('Job configuration created.')
    # Requesting the job
    load_job = bq.load_table_from_dataframe(
        df_procesado,
        table_id,
        location='us-central1',
        job_config=job_config
    )
    load_job.result()
    print('Job finished')


In [17]:
import pandas as pd


In [15]:
df=pd.read_csv(R"C:\Users\Usuario\Desktop\GitHub\DataSets\DataWarehouse\business.csv")

In [6]:
#df = pd.read_pickle(file_uri)
df=df.loc[:,~df.columns.duplicated()].copy()

In [11]:
df=df.sample(75000)

In [ ]:
import reverse_geocoder as rg
from tqdm import tqdm

def obtener_estado_y_ciudad(latitudes, longitudes):
    total_registros = len(latitudes)
    resultados = []
    
    # Inicializar barra de progreso
    with tqdm(total=total_registros, desc="Procesando") as pbar:
        for lat, lon in zip(latitudes, longitudes):
            coordinates = (lat, lon)
            result = rg.search(coordinates)  # Realiza la búsqueda inversa
            estado = result[0]['admin1']  # Obtiene el estado de la respuesta
            ciudad = result[0]['name']  # Obtiene el nombre de la ciudad de la respuesta
            resultados.append((estado, ciudad))
            pbar.update(1)  # Incrementar la barra de progreso
            pbar.set_postfix({"Completado": f"{pbar.n}/{pbar.total}"})  # Mostrar porcentaje completado
    
    return resultados

# Usar la función con las columnas de latitud y longitud del DataFrame
resultados = obtener_estado_y_ciudad(df['latitude'], df['longitude'])

# Crear columnas "state" y "city" con los resultados
df[['state', 'city']] = pd.DataFrame(resultados, columns=['state', 'city'])


codigo para corregir nombre de estados

In [23]:
import reverse_geocoder as rg
import pandas as pd
import time

def obtener_estado(ciudad):
    try:
        result = rg.search((ciudad, 0))  # Realiza la búsqueda inversa
        print(result)  # Imprimir el resultado de la búsqueda para depuración
        estado = result[0]['admin1']  # Obtiene el estado de la respuesta
        return estado
    except:
        return "No encontrado"

# Suponiendo que 'df' es tu DataFrame
inicio = time.time()  # Tiempo inicial
total_ciudades = len(df['city'])
for i, ciudad in enumerate(df['city'], 1):
    # Calcular el tiempo transcurrido y estimar el tiempo restante
    tiempo_transcurrido = time.time() - inicio
    tiempo_restante = (tiempo_transcurrido / i) * (total_ciudades - i)
    horas_restantes = int(tiempo_restante // 3600)
    minutos_restantes = int((tiempo_restante % 3600) // 60)
    segundos_restantes = int(tiempo_restante % 60)
    
    # Mostrar el progreso y el tiempo restante
    print(f"Procesando: {i}/{total_ciudades} ({100*i/total_ciudades:.2f}%) - Tiempo restante: {horas_restantes}h {minutos_restantes}m {segundos_restantes}s", end="\r")
    
    # Aplicar la función obtener_estado
    df.at[i-1, 'state'] = obtener_estado(ciudad)

fin = time.time()  # Tiempo final

# Calcular el tiempo total de ejecución
tiempo_total = fin - inicio

# Imprimir el tiempo total de ejecución
print(f"\nTiempo total de ejecución: {tiempo_total:.2f} segundos")

# Imprimir el DataFrame con las nuevas columnas
print(df)



Procesando: 150346/150346 (100.00%) - Tiempo restante: 0h 0m 0s
Tiempo total de ejecución: 27.20 segundos
                   business_id                      name  \
0       Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1       mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2       tUFrWirKiKi_TAnsVWINQQ                    Target   
3       MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4       mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
...                        ...                       ...   
150341  IUQopTMmYQG-qRtBk-8QnA              Binh's Nails   
150342  c8GjPIOTGVmIemT7j5_SyQ      Wild Birds Unlimited   
150343  _QAMST-NrQobXduilWEqSw         Claire's Boutique   
150344  mtGm22y5c2UHNXDFAjaPNw  Cyclery & Fitness Center   
150345  jV_XOycEzSlTx-65W906pg                   Sic Ink   

                                address           city          state  \
0                1616 Chapala St, Ste 2  Santa Barbara  No encontrado   
1       87 Grasso Plaza Sho

In [30]:
df.sample(20)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,is_open,categories
47179,jywqOvyPFJ4NT3KdcaNiGg,"Dennis M Lox, MD - Sports and Regenerative Med...",2030 Drew St,Clearwater,No encontrado,27.968249,-82.752792,4.0,7,1,"Health & Medical, Doctors, Pain Management, Ph..."
73073,edBkcpIZlX6tkXUc0A2e0Q,Courtyard Indianapolis at the Capitol,320 North Senate Avenue,Indianapolis,No encontrado,39.773010,-86.164672,3.5,38,1,"Hotels, Hotels & Travel, Event Planning & Serv..."
7262,LCYE3aQGQf74Z13U2g81Lg,Plymouth Auto & Tire Center,2014 Butler Pike,Plymouth Meeting,No encontrado,40.097040,-75.284217,2.0,71,1,"Automotive, Auto Repair"
7764,KVQQ0C3t0RJAUKeQK4kzkw,American Dental Solutions,545 W Main St,Trappe,No encontrado,40.200083,-75.478121,3.5,7,1,"Health & Medical, Dentists, Oral Surgeons, Gen..."
35238,cino46ny3O8RbGYZniVn5g,Infinite Endodontics,"1765 Springdale Rd, Ste C1",Cherry Hill,No encontrado,39.899651,-74.966712,3.0,14,0,"Endodontists, General Dentistry, Health & Medi..."
57977,LrZRyMmDbv065ZU4Czfgaw,Enterprise Rent-A-Car,706 Garrett Rd,Upper Darby,No encontrado,39.958017,-75.266992,2.5,11,1,"Hotels & Travel, Car Rental"
23556,4JzJG4KWRDFDROb8CjmFxQ,Motor Sheep,"5000 Smithridge Dr, Ste A7, Smithridge Shoppin...",Reno,No encontrado,39.478213,-119.789037,4.5,18,1,"Auto Upholstery, Furniture Reupholstery, Shopp..."
12057,s0MmNLweVTJsfmbGeqasag,Auto Tags & Insurance,702 MacDade Blvd,Collingdale,No encontrado,39.911974,-75.276561,5.0,6,1,"Financial Services, Auto Insurance, Motorcycle..."
138008,yX0P0_JV1imBE46I1ZMEaA,Panera Bread,"2534 Powell Ave, Ste 120",Nashville,No encontrado,36.106011,-86.763501,2.5,77,1,"Salad, Bagels, Soup, Breakfast & Brunch, Resta..."
24929,gQr86APLp8pmRVJlNIOvbQ,Waterway Carwash,10559 Old Olive St,Creve Coeur,No encontrado,38.674994,-90.412997,3.0,32,1,"Car Wash, Food, Gas Stations, Automotive, Conv..."


codigo que estoy probando con geo en cloud

In [ ]:
import functions_framework
import json
import pandas as pd 
import numpy as np 
from google.cloud import bigquery
from google.cloud import storage


# Importar modulo processing
#from processing import procesar, hash_category

# Instatiate clients
cloud_storage = storage.Client()
bq = bigquery.Client()

project_id = 'dms-pfh'


def archivo_df_business(file_uri):
    # Crear una instancia del cliente de Google Cloud Storage
    client = storage.Client()

    # Nombre del archivo en el bucket
    nombre_archivo = 'df_mascara.csv'

    # Nombre del bucket
    nombre_bucket = 'yelp_dms'
    # Ruta completa en el bucket
    ruta_archivo = f'gs://{nombre_bucket}/{nombre_archivo}'
    
    # Filtrado segun nombre archivo para su respectivo ETL
    df_bussines = pd.read_pickle(file_uri)
    df_bussines =df_bussines.loc[:,~df_bussines.columns.duplicated()].copy()
    print("pase la mascara de juan de columnas")
    df_bussines.drop(columns=["is_open",'attributes', 'hours','postal_code'], inplace=True)
    categorias_deseadas = ["restaurant", "bar", "pub", "cafe", 'bakery', 'coffee shop', 'gym', 'gas station']

    # Eliminar filas con valores NaN en la columna 'categories'
    df_bussines = df_bussines.fillna("No Data")
    # Filtrar el DataFrame para mantener solo las filas que contienen al menos una de las categorías deseadas
    df_bussines = df_bussines[df_bussines['categories'].str.contains('|'.join(categorias_deseadas), case=False)]
    df_mascara= df_bussines["business_id"]
    

    # Guardar el DataFrame como un archivo CSV
    df_mascara.to_csv(nombre_archivo, index=False)
    # Obtener el bucket
    bucket = client.bucket(nombre_bucket)
    # Subir el archivo al bucket
    blob = bucket.blob(nombre_archivo)
    blob.upload_from_filename(nombre_archivo)
    print(f'DataFrame guardado en {ruta_archivo}')


    return df_bussines

def archivo_df_tip(file_uri):
    df_tip = pd.read_json(file_uri, lines=True)
    df_tip.drop(columns="compliment_count", inplace=True)
    for columna in df_tip.columns:
        if df_tip[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_tip[columna] = df_tip[columna].fillna("No data").str.strip()
        elif df_tip[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_tip[columna] = df_tip[columna].fillna(0)
    return df_tip

def archivo_df_user(file_uri):
    df_user001 = pd.read_json(file_uri, lines=True)
    df_user001.drop(columns=['elite','friends','compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute','compliment_list', 'compliment_note', 'compliment_plain',
        'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos'], inplace=True)
    for columna in df_user001.columns:
        if df_user001[columna].dtype == 'object':
            # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
            df_user001[columna] = df_user001[columna].fillna("No data").str.strip()
        elif df_user001[columna].dtype in ['int64', 'float64']:
            # Si la columna es numérica, llenar los valores nulos con 0
            df_user001[columna] = df_user001[columna].fillna(0)
    print("completado etl archivo parquet")
    return df_user001

def archivo_df_review(file_uri):
    df_review = pd.read_json(file_uri, lines=True)
    #for columna in df_review.columns:
        #if df_review[columna].dtype == 'object':
        # Si la columna es de texto, llenar los valores nulos con "No data" y eliminar espacios en blanco y saltos de línea
        #    df_review[columna] = df_review[columna].fillna("No data").str.strip()
    #    if df_review[columna].dtype in ['int64', 'float64']:
                    # Si la columna es numérica, llenar los valores nulos con 0
    #        df_review[columna] = df_review[columna].fillna(0)
    df_mascara=pd.read_csv("gs://yelp_dms/df_mascara.csv")
    print("carga de mascara completada")
    df_review['business_id'] = df_review['business_id'].astype(str)
    df_mascara['business_id'] = df_mascara['business_id'].astype(str)

    df_review = pd.merge(df_review, df_mascara, on='business_id', how='inner')
    print("filtracion completa en archivo REVIEW")
    return df_review

# Triggered by a change in a storage bucket
@functions_framework.cloud_event
def load_df(cloud_event):
    data = cloud_event.data
    event_id = cloud_event["id"]
    event_type = cloud_event["type"]
    print(f'Cloud event ID: {event_id}')
    print(f'Event type: {event_type}')
    bucket_name = data["bucket"]
    name = data["name"]
    print(f'Bucket: {bucket_name}')
    print(f'File name: {name}')
    # Create URI
    file_uri = f'gs://{bucket_name}/{name}'
    print(f'URI: {file_uri}')
    # Obtener el objeto del bucket
    bucket = cloud_storage.bucket(bucket_name)
    # Obtener el blob (objeto) del archivo JSON en el bucket
    blob = bucket.blob(name)
    # Attempt to create DataFrame with URI
    # Attempt to process DataFrame
    print('Transformando dataframe...')
    try:
        if "business" in file_uri:
            df_procesado = archivo_df_business(file_uri)
            table_name = "yelp-sites"
            print("dataframe cargado") 
        if "tip" in file_uri:
            df_procesado = archivo_df_tip(file_uri)
            table_name = "yelp-user-tips"
            print("dataframe cargado")
        if "user" in file_uri:
            df_procesado = archivo_df_user(file_uri)
            table_name = "yelp-user-info"
            print("dataframe cargado")
        if "parte_" in file_uri:
            df_procesado=archivo_df_review(file_uri)
            table_name="yelp-user-review"
            print("dataframe cargado")
        print('DataFrame Procesado')
        print('Nuevas dimensiones:', df_procesado.shape)
    except Exception as e:
        # Cuando no procesa la información
        print('DataFrame no pudo ser procesado: Error - ', e)
        return None
    # Insert to table
    table_id = f'dms-pfh.yelp_data.{table_name}'
    # Create job configurations
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        create_disposition='CREATE_IF_NEEDED',
        write_disposition='WRITE_APPEND'
        # source_format='NEWLINE_DELIMITED_JSON'
    )
    print('Job configuration created.')
    # Requesting the job
    load_job = bq.load_table_from_dataframe(
        df_procesado,
        table_id,
        location='us-central1',
        job_config=job_config
    )
    load_job.result()
    print('Job finished')
